# Estrategia: Precio Bajo con Filtros ADX y Volumen - Tuning

## Objetivo
Este notebook tiene como objetivo afinar los parámetros de la estrategia `precio_bajo_filtros_v1`, que busca generar señales de compra cuando el precio está en una zona baja relativa dentro de una ventana temporal, utilizando filtros opcionales de tendencia (ADX) y volumen.

## Parámetros configurables

- `window`: Tamaño de la ventana para evaluar si el precio está en la parte baja del rango. (ej: 50, 100, 150)
- `adx_threshold`: Umbral mínimo de fuerza de tendencia (ADX) requerido para validar la señal.
- `usar_adx`: Activa o desactiva el filtro de tendencia.
- `usar_volumen`: Activa o desactiva el filtro de volumen institucional.

## Resultado esperado
Encontrar las combinaciones de parámetros que maximizan el rendimiento de la estrategia en función de métricas como profit promedio, winrate y score.


# 🧠 Explicación general de la estrategia y objetivo del notebook
----------------------------------------------------------------
## Estrategia: Precio Bajo con Filtros ADX y Volumen - v1

## Objetivo:
Esta notebook ejecuta un proceso de tuning sobre parámetros clave de la estrategia,
cuyo propósito es identificar zonas de compra cuando el precio se encuentra en la
parte baja de una ventana temporal, usando filtros de tendencia y volumen para mejorar la calidad de señales.

## Parámetros:
- window: Tamaño de la ventana para calcular mínimos/máximos.
- adx_threshold: Umbral mínimo de fuerza direccional (ADX) para validar señales.
- usar_adx: Activar o desactivar filtro de tendencia.
- usar_volumen: Activar o desactivar filtro de volumen.

La notebook realiza tuning en paralelo y reporta las combinaciones más efectivas.


In [1]:
import os
import pandas as pd

ruta_historicos = "D:/trading/data/historic"
historicos = {}

for archivo in os.listdir(ruta_historicos):
    if archivo.endswith(".parquet"):
        ticker = archivo.replace(".parquet", "")
        df = pd.read_parquet(os.path.join(ruta_historicos, archivo))
        historicos[ticker] = df

print(f"✅ Símbolos cargados: {len(historicos)}")

✅ Símbolos cargados: 48


In [2]:
import itertools

valores_window = [50, 75, 100]
valores_adx = [10, 15, 20]
valores_adx_on = [True]
valores_vol_on = [True, False]

grid_parametros = list(itertools.product(valores_window, valores_adx, valores_adx_on, valores_vol_on))
print(f"Total combinaciones: {len(grid_parametros)}")

Total combinaciones: 18


In [3]:
import sys
sys.path.append("D:/trading")

from my_modules.estrategias.v1.precio_bajo_filtros_v1 import generar_senales

def simular_combinacion(param_combo, historicos):
    window, adx_threshold, usar_adx, usar_volumen = param_combo
    resumenes = []

    for simbolo, df in historicos.items():
        try:
            df_signals = generar_senales(
                df,
                window=window,
                adx_threshold=adx_threshold,
                usar_adx=usar_adx,
                usar_volumen=usar_volumen,
                debug=False
            )

            df_signals = df_signals.merge(df[["fecha", "close"]], on="fecha", how="left")
            df_signals["retorno"] = df_signals["close"].pct_change().shift(-1)  # Asumimos ejecución al cierre

            df_signals = df_signals[df_signals["signal"] == "buy"]
            profit_mean = df_signals["retorno"].mean()
            winrate = (df_signals["retorno"] > 0).mean()
            n_trades = len(df_signals)

            resumenes.append({
                "symbol": simbolo,
                "window": window,
                "adx_threshold": adx_threshold,
                "usar_adx": usar_adx,
                "usar_volumen": usar_volumen,
                "profit_mean": profit_mean,
                "winrate": winrate,
                "n_trades": n_trades
            })
        except Exception as e:
            print(f"{simbolo}: {e}")
    
    return resumenes

In [4]:
from joblib import Parallel, delayed
from tqdm import tqdm

def ejecutar_tuning(historicos, combinaciones):
    def safe_simulacion(comb):
        try:
            return simular_combinacion(comb, historicos)
        except Exception as e:
            print(f"Error en combinación {comb}: {e}")
            return []

    resultados = Parallel(n_jobs=-1, backend="loky")(
        delayed(safe_simulacion)(comb) for comb in tqdm(combinaciones)
    )

    # Aplanar lista de listas
    planos = [item for sublist in resultados for item in sublist]
    return pd.DataFrame(planos)

# Ejecutar tuning
df_resultados = ejecutar_tuning(historicos, grid_parametros)

# Mostrar top
print("Top combinaciones por score:")
display(df_resultados.sort_values("profit_mean", ascending=False).head(10))

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.67it/s]


Top combinaciones por score:


,symbol,window,adx_threshold,usar_adx,usar_volumen,profit_mean,winrate,n_trades
775,CTVA,100,20,True,True,0.012938,0.666667,45
199,CTVA,50,20,True,True,0.012660,0.650000,60
679,CTVA,100,15,True,True,0.012229,0.644068,59
819,AVGO,100,20,True,False,0.012219,0.635294,85
771,AVGO,100,20,True,True,0.012208,0.627907,43
583,CTVA,100,10,True,True,0.010736,0.634921,63
487,CTVA,75,20,True,True,0.010726,0.625000,56
579,AVGO,100,10,True,True,0.010253,0.606557,61
772,BABA,100,20,True,True,0.010225,0.602151,93
723,AVGO,100,15,True,False,0.010117,0.605769,104


In [5]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df_resultados["score"].values, marker="o")
plt.title("Distribución de Score por Configuración Breakout Volumen")
plt.xlabel("Ranking")
plt.ylabel("Score")
plt.grid(True)
plt.tight_layout()
plt.show()

KeyError: 'score'

<Figure size 1000x600 with 0 Axes>